# Common Transformations

Apache Spark&trade; and Azure Databricks&reg; allow you to manipulate data with built-in functions that accommodate common design patterns.

-sandbox
### Transformations in ETL

The goal of transformations in ETL is to transform raw data in order to populate a data model.  The most common models are **relational models** and **snowflake (or star) schemas,** though other models such as query-first modeling also exist. 

Transforming data can range in complexity from simply parsing relavant fields to handling null values without affecting downstream operations and applying complex conditional logic.  Common transformations include:<br><br>

* Normalizing values
* Imputing null or missing data
* Deduplicating data
* Performing database rollups
* Exploding arrays
* Pivoting DataFrames

<div><img src="https://files.training.databricks.com/images/eLearning/ETL-Part-2/data-models.png" style="height: 400px; margin: 20px"/></div>

-sandbox
### Built-In Functions

Built-in functions offer a range of performant options to manipulate data. This includes options familiar to:<br><br>

1. SQL users such as `.select()` and `.groupBy()`
2. Python, Scala and R users such as `max()` and `sum()`
3. Data warehousing options such as `rollup()` and `cube()`

-sandbox
### Getting Started

Run the following cell to configure our "classroom."

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Remember to attach your notebook to a cluster. Click <b>Detached</b> in the upper left hand corner and then select your preferred cluster.

<img src="https://files.training.databricks.com/images/eLearning/attach-to-cluster.png" style="border: 1px solid #aaa; border-radius: 10px 10px 10px 10px; box-shadow: 5px 5px 5px #aaa"/>

Run the cell below to mount the data.

In [6]:
%run "./Includes/Classroom-Setup"

### Normalizing Data

Normalizing refers to different practices including restructuring data in normal form to reduce redundancy, and scaling data down to a small, specified range. For this case, bound a range of integers between 0 and 1.

Start by taking a DataFrame of a range of integers

In [9]:
integerDF = spark.range(1000, 10000)

display(integerDF)

id
1000
1001
1002
1003
1004
1005
1006
1007
1008
1009


-sandbox

To normalize these values between 0 and 1, subtract the minimum and divide by the maximum, minus the minimum.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** <a href="http://spark.apache.org/docs/latest/api/python/pyspark.ml.html?highlight=minmaxscaler#pyspark.ml.feature.MinMaxScaler" target="_blank">Also see the bult-in function `MinMaxScaler`</a>

In [11]:
from pyspark.sql.functions import col, max, min

colMin = integerDF.select(min("id")).first()[0]
colMax = integerDF.select(max("id")).first()[0]

normalizedIntegerDF = (integerDF
  .withColumn("normalizedValue", (col("id") - colMin) / (colMax - colMin) )
)

display(normalizedIntegerDF)

id,normalizedValue
1000,0.0
1001,1.1112345816201801E-4
1002,2.2224691632403602E-4
1003,3.33370374486054E-4
1004,4.4449383264807203E-4
1005,5.5561729081009E-4
1006,6.66740748972108E-4
1007,7.77864207134126E-4
1008,8.889876652961441E-4
1009,0.001000111123458162


-sandbox
### Imputing Null or Missing Data

Null values refer to unknown or missing data as well as irrelevant responses. Strategies for dealing with this scenerio include:<br><br>

* **Dropping these records:** Works when you do not need to use the information for downstream workloads
* **Adding a placeholder (e.g. `-1`):** Allows you to see missing data later on without violating a schema
* **Basic imputing:** Allows you to have a "best guess" of what the data could have been, often by using the mean of non-missing data
* **Advanced imputing:** Determines the "best guess" of what data should be using more advanced strategies such as clustering machine learning algorithms or oversampling techniques 

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** <a href="http://spark.apache.org/docs/latest/api/python/pyspark.ml.html?highlight=imputer#pyspark.ml.feature.Imputer" target="_blank">Also see the bult-in function `Imputer`</a>

Take a look at the following DataFrame, which has missing values.

In [14]:
corruptDF = spark.createDataFrame([
  (11, 66, 5),
  (12, 68, None),
  (1, None, 6),
  (2, 72, 7)], 
  ["hour", "temperature", "wind"]
)

display(corruptDF)

hour,temperature,wind
11,66,5
12,68,null
1,null,6
2,72,7


Drop any records that have null values.

In [16]:
corruptDroppedDF = corruptDF.dropna("any")

display(corruptDroppedDF)

hour,temperature,wind
11,66,5
2,72,7


Impute values with the mean.

In [18]:
from pyspark.sql.functions import col, min, avg
import pyspark.sql.functions as F

corruptImputedDF = corruptDF.na.fill({"temperature": 68, "wind": 6})
#corruptImputedDF = corruptDF.na.fill({"temperature": F.avg(col("corruptDroppedDF.temperature")), "wind": F.avg(col("corruptDroppedDF.wind"))})

display(corruptImputedDF)

hour,temperature,wind
11,66,5
12,68,6
1,68,6
2,72,7


### Deduplicating Data

Duplicate data comes in many forms. The simple case involves records that are complete duplicates of another record. The more complex cases involve duplicates that are not complete matches, such as matches on one or two columns or "fuzzy" matches that account for formatting differences or other non-exact matches.

Take a look at the following DataFrame that has duplicate values.

In [21]:
duplicateDF = spark.createDataFrame([
  (15342, "Conor", "red"),
  (15342, "conor", "red"),
  (12512, "Dorothy", "blue"),
  (5234, "Doug", "aqua")], 
  ["id", "name", "favorite_color"]
)

display(duplicateDF)

id,name,favorite_color
15342,Conor,red
15342,conor,red
12512,Dorothy,blue
5234,Doug,aqua


Drop duplicates on `id` and `favorite_color`.

In [23]:
duplicateDedupedDF = duplicateDF.dropDuplicates(["id", "favorite_color"])
#duplicateDedupedDF = duplicateDF.dropDuplicates()

display(duplicateDedupedDF)

id,name,favorite_color
5234,Doug,aqua
12512,Dorothy,blue
15342,Conor,red


### Other Helpful Data Manipulation Functions

| Function    | Use                                                                                                                        |
|:------------|:---------------------------------------------------------------------------------------------------------------------------|
| `explode()` | Returns a new row for each element in the given array or map                                                               |
| `pivot()`   | Pivots a column of the current DataFrame and perform the specified aggregation                                             |
| `cube()`    | Create a multi-dimensional cube for the current DataFrame using the specified columns, so we can run aggregation on them   |
| `rollup()`  | Create a multi-dimensional rollup for the current DataFrame using the specified columns, so we can run aggregation on them |

## Exercise 1: Deduplicating Data

A common ETL workload involves cleaning duplicated records that don't completely match up.  The source of the problem can be anything from user-generated content to schema evolution and data corruption.  Here, you match records and reduce duplicate records.

-sandbox
### Step 1: Import and Examine the Data

The file is sitting in `/mnt/training/dataframes/people-with-dups.txt`.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** You have to deal with the header and delimiter.

In [27]:
# TODO
dupedDF = (spark.read
   .option("header", True)
   .option("inferSchema", True)
   .option("delimiter", ":")
   .csv("/mnt/training/dataframes/people-with-dups.txt")
)

display(dupedDF)

firstName,middleName,lastName,gender,birthDate,salary,ssn
Emanuel,Wallace,Panton,M,1988-03-04T00:00:00.000+0000,101255,935-90-7627
Eloisa,Rubye,Cayouette,F,2000-06-20T00:00:00.000+0000,204031,935-89-9009
Cathi,Svetlana,Prins,F,2012-12-22T00:00:00.000+0000,35895,959-30-7957
Mitchel,Andres,Mozdzierz,M,1966-05-06T00:00:00.000+0000,55108,989-27-8093
Angla,Melba,Hartzheim,F,1938-07-26T00:00:00.000+0000,13199,935-27-4276
Rachel,Marlin,Borremans,F,1923-02-23T00:00:00.000+0000,67070,996-41-8616
Catarina,Phylicia,Dominic,F,1969-09-29T00:00:00.000+0000,201021,999-84-8888
Antione,Randy,Hamacher,M,2004-03-05T00:00:00.000+0000,271486,917-96-3554
Madaline,Shawanda,Piszczek,F,1996-03-17T00:00:00.000+0000,183944,963-87-9974
Luciano,Norbert,Sarcone,M,1962-12-14T00:00:00.000+0000,73069,909-96-1669


In [28]:
%fs head /mnt/training/dataframes/people-with-dups.txt

[Truncated to first 65536 bytes]
firstName:middleName:lastName:gender:birthDate:salary:ssn
Emanuel:Wallace:Panton:M:1988-03-04:101255:935-90-7627
Eloisa:Rubye:Cayouette:F:2000-06-20:204031:935-89-9009
Cathi:Svetlana:Prins:F:2012-12-22:35895:959-30-7957
Mitchel:Andres:Mozdzierz:M:1966-05-06:55108:989-27-8093
Angla:Melba:Hartzheim:F:1938-07-26:13199:935-27-4276
Rachel:Marlin:Borremans:F:1923-02-23:67070:996-41-8616
Catarina:Phylicia:Dominic:F:1969-09-29:201021:999-84-8888
Antione:Randy:Hamacher:M:2004-03-05:271486:917-96-3554
Madaline:Shawanda:Piszczek:F:1996-03-17:183944:963-87-9974
Luciano:Norbert:Sarcone:M:1962-12-14:73069:909-96-1669
Newton:Jose:Piacente:M:1969-11-05:264422:967-61-8575
Gretta:Jennefer:Dipinto:F:1921-10-28:118497:907-49-2510
Jenni:Janella:Mcquiston:F:1969-07-11:137418:932-55-7164
Malena:Apryl:Kings:F:1980-01-08:98204:934-48-2334
Willy:Russell:Merker:M:1967-10-10:41026:992-83-4829
Jami:Allen:Mulkerin:F:1958-08-08:236024:934-86-4126
Fernando:Lowell:Zebell:M:1955-07-03:246516:951-57-6196
Garnet:Helaine:Edghill:F:1985-05-15:97125:979-42-4409
Siu:Cherrie:Lechelt:F:2012-07-24:148331:906-85-3202
Randal:Osvaldo:Torma:M:2013-12-17:275086:982-68-6906
Myles:Stuart:Miskell:M:1928-05-17:280726:992-89-2715
Conrad:Donnie:Piatkowski:M:1920-11-17:59935:916-33-7831
Darcel:Fleta:Czyrnik:F:1994-09-05:99641:995-95-8909
Juliane:Hallie:Vuillemot:F:1972-01-11:203415:950-17-2657
Cordia:Estella:Scheiber:F:1924-03-13:214853:965-74-4739
Bryon:Isiah:Nazario:M:2002-02-26:201478:963-38-1755
Alex:Gilbert:Trigueros:M:1990-12-30:271591:999-60-3232
Irwin:Rene:Zalesky:M:1931-06-20:122599:974-53-6387
Deandre:Brent:Bailor:M:1954-08-07:93429:920-78-9780
Cheree:Dorethea:Anspaugh:F:1985-01-17:278860:961-36-6578
Zachariah:Harris:Reemer:M:1924-12-16:238892:944-32-3013
See:Sharen:Howryla:F:1979-12-30:169570:925-12-1644
Mozella:Necole:Giannakopoulo:F:1993-08-24:40880:980-93-4844
David:Tracy:Delatrinidad:M:1932-01-26:85003:993-37-2066
Michel:Loren:Sandoz:M:1925-05-04:270046:948-56-4284
Graham:Denny:Apresa:M:1966-07-14:209118:928-67-3592
Kattie:Sammie:Ercek:F:2002-07-26:211993:996-32-1564
Shaunna:Kathlene:Amunrud:F:1965-07-01:234090:916-53-6216
Leeann:Madeleine:Clater:F:1976-10-26:100705:946-41-7003
Garry:Lino:Galm:M:1934-08-14:168769:994-29-1678
Shelly:Marhta:Krenzke:F:1915-05-08:270492:967-71-8565
Shantelle:Zoe:Spicher:F:1952-12-11:183419:989-79-8274
Yessenia:Lucina:Fred:F:1967-04-21:131376:990-95-7077
Del:Maynard:Dzurnak:M:1960-01-31:276319:938-46-9967
Maximina:Kirsten:Barranco:F:2004-11-05:257482:938-70-3631
Shasta:Bette:Licausi:F:2006-05-26:273311:929-84-1247
Wendell:Bud:Landi:M:2012-03-06:61215:987-95-9502
Sabine:Petronila:Lambertson:F:1937-03-31:274044:916-95-8546
Leonard:Filiberto:Sellberg:M:1982-05-25:264070:965-65-6038
Demetrice:Flossie:Schehr:F:1958-03-16:245090:990-42-7824
Reyes:Gilberto:Schlabs:M:1997-07-11:252679:906-83-6847
Bernard:Reggie:Coache:M:1960-06-23:53020:941-56-6401
Lavette:Elouise:Riedinger:F:1950-11-21:51371:913-81-9468
Edmund:Nicolas:Majocka:M:1974-05-26:298673:921-55-7816
Deana:Marcie:Malle:F:2011-06-24:15388:973-52-9173
Claudio:Leopoldo:Ealand:M:1916-01-14:158319:929-61-6825
Lucretia:Divina:Parnell:F:2011-05-30:41054:963-43-3135
Myrtis:Yaeko:Maciarello:F:1941-11-16:191903:996-58-7685
Cordie:Cara:Sheilds:F:2007-02-08:219449:950-98-5411
Michell:Corinne:Himmelspach:F:1968-02-27:154658:905-67-9613
Major:Shawn:Garay:M:1934-12-18:79152:904-49-5993
Andrew:Darnell:Kniceley:M:1966-09-02:25955:987-26-4880
Shasta:Floria:Messineo:F:2008-02-23:30339:925-83-2833
Madalene:Arline:Vanholland:F:1945-05-14:47734:977-68-6192
Clare:So:Bieker:F:1922-04-07:14603:922-40-9560
Audrey:Lorrine:Sprewell:F:1932-10-25:283164:997-53-7925
ARLEN:HAYDEN:CARVILL:M:1986-05-24:66754:918339442
Maybelle:Honey:Jolicoeur:F:1923-02-10:239772:902-46-1282
Darwin:Cedric:Gompf:M:1954-03-19:152991:926-88-9050
Jonelle:Maryam:Berkstresser:F:1944-11-17:253619:935-62-7974
Lynsey:Marcelina:Dagrella:F:1969-03-01:209989:978-42-7296
Robert:Jessie:Anaya:M:1922-09-30:136381:956-53-6

In [29]:
# TEST - Run this cell to test your solution
cols = set(dupedDF.columns)

dbTest("ET2-P-02-01-01", 103000, dupedDF.count())
dbTest("ET2-P-02-01-02", True, "salary" in cols and "lastName" in cols)

print("Tests passed!")

Tests passed!

### Step 2: Add Columns to Filter Duplicates

Add columns following to allow you to filter duplicate values.  Add the following:

- `lcFirstName`: first name lower case
- `lcLastName`: last name lower case
- `lcMiddleName`: middle name lower case
- `ssnNums`: social security number without hyphens between numbers

Save the results to `dupedWithColsDF`.

In [31]:
# TODO
from pyspark.sql.functions import col, lower, translate
dupedWithColsDF = (dupedDF
                   .withColumn("lcFirstName", lower(col("firstName")))
                   .withColumn("lcLastName", lower(col("lastName")))
                   .withColumn("lcMiddleName", lower(col("middleName")))
                   .withColumn("snsNum", translate(col("ssn"), "-", ""))
                  )
display(dupedWithColsDF)                   # FILL_IN

firstName,middleName,lastName,gender,birthDate,salary,ssn,lcFirstName,lcLastName,lcMiddleName,snsNum
Emanuel,Wallace,Panton,M,1988-03-04T00:00:00.000+0000,101255,935-90-7627,emanuel,panton,wallace,935907627
Eloisa,Rubye,Cayouette,F,2000-06-20T00:00:00.000+0000,204031,935-89-9009,eloisa,cayouette,rubye,935899009
Cathi,Svetlana,Prins,F,2012-12-22T00:00:00.000+0000,35895,959-30-7957,cathi,prins,svetlana,959307957
Mitchel,Andres,Mozdzierz,M,1966-05-06T00:00:00.000+0000,55108,989-27-8093,mitchel,mozdzierz,andres,989278093
Angla,Melba,Hartzheim,F,1938-07-26T00:00:00.000+0000,13199,935-27-4276,angla,hartzheim,melba,935274276
Rachel,Marlin,Borremans,F,1923-02-23T00:00:00.000+0000,67070,996-41-8616,rachel,borremans,marlin,996418616
Catarina,Phylicia,Dominic,F,1969-09-29T00:00:00.000+0000,201021,999-84-8888,catarina,dominic,phylicia,999848888
Antione,Randy,Hamacher,M,2004-03-05T00:00:00.000+0000,271486,917-96-3554,antione,hamacher,randy,917963554
Madaline,Shawanda,Piszczek,F,1996-03-17T00:00:00.000+0000,183944,963-87-9974,madaline,piszczek,shawanda,963879974
Luciano,Norbert,Sarcone,M,1962-12-14T00:00:00.000+0000,73069,909-96-1669,luciano,sarcone,norbert,909961669


In [32]:
# TEST - Run this cell to test your solution
cols = set(dupedWithColsDF.columns)

dbTest("ET2-P-02-02-01", 103000, dupedWithColsDF.count())
dbTest("ET2-P-02-02-02", True, "lcFirstName" in cols and "lcLastName" in cols)

print("Tests passed!")

Tests passed!

### Step 3: Deduplicate the Data

Deduplicate the data by dropping duplicates of all records except for the original names (first, middle, and last) and the original `ssn`.  Save the result to `dedupedDF`.  Drop the columns you added in step 2.

In [34]:
# TODO
cols = dupedWithColsDF.columns
org_name_cols = ["firstName", "middleName", "lastName"]
[cols.remove(x) for x in org_name_cols]
added_cols = "lcFirstName", "lcMiddleName", "lcLastName", "snsNum"

dedupedDF = dupedWithColsDF.dropDuplicates(["lcFirstName", "lcMiddleName", "lcLastName", "snsNum", "gender", "birthDate", "salary"]).drop("lcFirstName", "lcMiddleName", "lcLastName", "snsNum")# FILL_IN

In [35]:
cols = dupedWithColsDF.columns
org_name_cols = ["firstName", "middleName", "lastName"]
[cols.remove(x) for x in org_name_cols]
cols

Out[ 48 ]: ['gender',
 'birthDate',
 'salary',
 'ssn',
 'lcFirstName',
 'lcLastName',
 'lcMiddleName',
 'snsNum']

In [36]:
# TEST - Run this cell to test your solution
cols = set(dedupedDF.columns)

dbTest("ET2-P-02-03-01", 100000, dedupedDF.count())
dbTest("ET2-P-02-03-02", 7, len(cols))

print("Tests passed!")

Tests passed!

## Review
**Question:** What built-in functions are available in Spark?  
**Answer:** Built-in functions include SQL functions, common programming language primitives, and data warehousing specific functions.  See the Spark API Docs for more details. (<a href="http://spark.apache.org/docs/latest/api/python/index.html" target="_blank">Python</a> or <a href="http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.package" target="_blank">Scala</a>).

**Question:** What's the best way to handle null values?  
**Answer:** The answer depends largely on what you hope to do with your data moving forward. You can drop null values or impute them with a number of different techniques.  For instance, clustering your data to fill null values with the values of nearby neighbors often gives more insight to machine learning models than using a simple mean.

**Question:** What are potential challenges of deduplicating data and imputing null values?  
**Answer:** Challenges include knowing which is the correct record to keep and how to define logic that applies to the root cause of your situation. This decision making process depends largely on how removing or imputing data will affect downstream operations like database queries and machine learning workloads. Knowing the end application of the data helps determine the best strategy to use.

## Next Steps

Start the next lesson, [User Defined Functions]($./03-User-Defined-Functions ).

## Additional Topics & Resources

**Q:** How can I do ACID transactions with Spark?  
**A:** ACID compliance refers to a set of properties of database transactions that guarantee the validity of you data.  <a href="https://databricks.com/product/databricks-delta" target="_blank">Databricks Delta</a> is an ACID compliant solution to transactionality with Spark workloads.

**Q:** How can I handle more complex conditional logic in Spark?  
**A:** You can handle more complex if/then conditional logic using the `when()` function and its `.otherwise()` method.

**Q:** How can I handle data warehousing functions like rollups?  
**A:** Spark allows for rollups and cubes, which are common in star schemas, using the `rollup()` and `cube()` functions.